In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup
import scrapy
import pandas as pd
import csv
import os
import requests
import time
import random 
from datetime import datetime


In [2]:
driver = webdriver.Chrome('./chromedriver.exe')
driver.get('https://ebird.org/explore')
input_ebird = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[placeholder='Enter species name']")))


C:\Users\LEGION\AppData\Local\Temp\ipykernel_39056\4268123770.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe')


In [3]:
input_ebird.send_keys('Mourning dove')
input_ebird.send_keys(Keys.ENTER)
time.sleep(2)
input_ebird.send_keys(Keys.ENTER)

In [ ]:
url = driver.current_url

In [19]:
import requests
from bs4 import BeautifulSoup

# placeholder="Enter species name" 
def get_page_of_bird_ebird(bird_name,driver):
    driver.get('https://ebird.org/explore')
    input_ebird = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[placeholder='Enter species name']")))
    input_ebird.send_keys(bird_name)
    input_ebird.send_keys(Keys.ENTER)
    time.sleep(2)
    input_ebird.send_keys(Keys.ENTER)
    return driver.current_url

# URL = "https://ebird.org/species/grswoo"
def scrape(URL):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    results = soup.find_all("div",class_="Breadcrumbs")
    names_main = soup.find_all("span",class_="Heading-main")
    names_sub = soup.find_all("span",class_="Heading-sub")
    udescription=soup.find_all("p",class_="u-stack-sm")
    for i,j,k,l in zip(results,names_main,names_sub,udescription):
        print('fam , order :',i.get_text())
        print('Main :',j.get_text())
        print('Sci name :',k.get_text())
        print('des : ',l.get_text())
# scrape(URL)
def scrape_v_2_ebird(URL):
    if URL :
        d = {}
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find_all("div",class_="Breadcrumbs")
        names_main = soup.find_all("span",class_="Heading-main")
        names_sub = soup.find_all("span",class_="Heading-sub")
        udescription=soup.find_all("p",class_="u-stack-sm")
        
        for i,j,k,l in zip(results,names_main,names_sub,udescription):
            a = i.get_text().strip().split('\n')
            b = j.get_text().strip()
            c = k.get_text().strip()
            e = l.get_text()
            d[b] = {'Order':a[0],'Family':a[1],'Scientific Name':c,'Description':e}
        return d
    return 'NODATA'


def ebird(fro,to,csv_file,driver):
    types_of_birds_df = pd.read_csv('assets/birds.csv',index_col='ID')
    list_of_bird_names = [ types_of_birds_df.loc[ _ ,'Common Bird Names'] for _ in range(1,types_of_birds_df.shape[0]) ]
    data_from_ebird = {}
    re_search_on = [] 
    writer = csv.writer(open('assets/'+ csv_file +'.csv',"a",newline=''))
    for bird_name in list_of_bird_names[fro:to]:
        # try:
        data_from_ebird[bird_name] = scrape_v_2_ebird(get_page_of_bird_ebird(bird_name,driver))
        try:
            writer.writerow([bird_name,data_from_ebird[bird_name]])
        except UnicodeEncodeError:
            writer.writerow([list_of_bird_names.index(bird_name),data_from_ebird[bird_name]])
        if (not data_from_ebird) or (data_from_ebird[bird_name]=='NODATA') :
            re_search_on.append(bird_name)
            print('[-] ',bird_name)
        else:
            print('[+] ',bird_name)
        # except Exception:
            # writer.writerow([bird_name,'NODATA'])
            # print('----------> Unknown Exceptions for Bird ',bird_name)
    print(re_search_on)

# driver = webdriver.Chrome('./chromedriver.exe')
# get_page_of_bird_ebird('parrot',driver)

In [21]:
def main_ebird(f,t,ds):
    startTime = datetime.now()
    driver = webdriver.Chrome('./chromedriver.exe')
    ebird(f,t,ds,driver)
    print('\n\nTime taken : ',datetime.now() - startTime)
    driver.quit()
main_ebird(1,3,'ebird/ebird_data')

C:\Users\LEGION\AppData\Local\Temp\ipykernel_39056\906290530.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe')


[+]  Abbott's booby
[+]  Abbott's starling
[]


Time taken :  0:00:25.758294


In [23]:
def scrape_v_2_ebird(URL):
    d = {}
    if URL :
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find_all("div",class_="Breadcrumbs")
        names_main = soup.find_all("span",class_="Heading-main")
        names_sub = soup.find_all("span",class_="Heading-sub")
        udescription=soup.find_all("p",class_="u-stack-sm")
        Conservation = soup.find_all("span",class_="Badge u-inline-xs u-color-constatus-lc")
        # birds = pd.DataFrame(columns=['Common Name','Conservation status','Kingdom','Phylum','Class','Order','Family', 'Binomial Name', 'Link'])
        for i,j,k,l,m in zip(results,names_main,names_sub,udescription,Conservation):
            a = i.get_text().strip().split('\n')
            b = j.get_text().strip()
            c = k.get_text().strip()
            e = l.get_text()
            s = m.get_text().strip().split('\n')
            d[b] = {'Order':a[0],'Family':a[1],'Scientific Name':c,'Description':e,'status':s,'URL':URL}
    return d
scrape_v_2_ebird('https://ebird.org/species/grswoo')

{}

FileNotFoundError: [Errno 2] No such file or directory: 'Pick_ebird_1_1500.pkl'